## Enviroment setup

In [11]:
#@title Setup & Environment Verification

import warnings
warnings.filterwarnings('ignore')

import os
import sys

print("=== ENVIRONMENT CHECK ===")
print(f"Python: {sys.version.split()[0]}")
print(f"JAVA_HOME: {os.environ.get('JAVA_HOME')}")
print(f"SPARK_HOME: {os.environ.get('SPARK_HOME')}")
print(f"Driver Memory: {os.environ.get('SPARK_DRIVER_MEMORY')}")
print(f"Executor Memory: {os.environ.get('SPARK_EXECUTOR_MEMORY')}")
print("=" * 50)

=== ENVIRONMENT CHECK ===
Python: 3.10.12
JAVA_HOME: /usr/lib/jvm/java-8-openjdk-arm64/jre
SPARK_HOME: /opt/spark
Driver Memory: 12g
Executor Memory: 8g


In [12]:
#@title Import Libraries

# PySpark
from pyspark import SparkContext
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, HashingTF, IDF, StringIndexer
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window

# SciKit Learn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# TensorFlow
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Dropout, Bidirectional, GlobalMaxPooling1D, Concatenate, SpatialDropout1D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

# Data manipulation
import pandas as pd
import numpy as np

# Financial data
import yfinance as yf

# Hugging Face
from huggingface_hub import hf_hub_download

# Kaggle
import kagglehub

# Plots
import matplotlib.pyplot as plt

# Utilities
from datetime import datetime, timedelta
import json
import requests
import logging
from tqdm import tqdm
import time
import subprocess
from pathlib import Path
from collections import Counter
import pickle
import re

In [13]:
#@title Start Spark session

print("=== PRE-FLIGHT CHECK ===")

# Verify Java is available
try:
    java_version = subprocess.check_output(['java', '-version'], stderr=subprocess.STDOUT)
    print("Java: ✅ Available")
except Exception as e:
    print(f"Java: ❌ Not available - {e}")

print("=" * 50)

# 🔥 STOP any existing Spark sessions first
try:
    SparkContext.getOrCreate().stop()
    print("🧹 Cleaned up existing Spark session")
except:
    print("🆕 No existing session to clean")

print("=" * 50)

# Create fresh Spark session
spark = SparkSession.builder \
    .appName("Yelp_Sentiment_Analysis") \
    .master("local[*]") \
    .config("spark.driver.memory", "12g") \
    .config("spark.executor.memory", "8g") \
    .config("spark.driver.maxResultSize", "4g") \
    .config("spark.memory.fraction", "0.8") \
    .config("spark.memory.storageFraction", "0.3") \
    .config("spark.sql.shuffle.partitions", "200") \
    .config("spark.default.parallelism", "16") \
    .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .config("spark.kryoserializer.buffer.max", "512m") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .getOrCreate()

spark.sparkContext.setLogLevel("WARN")

print("✅ Spark session configured with:")
print(f"   - Driver Memory: 12GB")
print(f"   - Executor Memory: 8GB")
print(f"   - Max Result Size: 4GB")
print(f"   - Parallelism: 16 cores")
print(f"   - Shuffle Partitions: 200")

=== PRE-FLIGHT CHECK ===
Java: ✅ Available
🧹 Cleaned up existing Spark session
✅ Spark session configured with:
   - Driver Memory: 12GB
   - Executor Memory: 8GB
   - Max Result Size: 4GB
   - Parallelism: 16 cores
   - Shuffle Partitions: 200


25/11/07 15:45:42 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


## Predictions

In [14]:
print("=" * 80)
print("🤖 LOADING LSTM SENTIMENT CLASSIFIER")
print("=" * 80)

model = tf.keras.models.load_model('../models/lstm_sentiment_best.keras')

with open('../data/processed/word_to_index.pkl', 'rb') as f:
    word_to_index = pickle.load(f)

with open('../data/processed/label_encoder.pkl', 'rb') as f:
    label_encoder = pickle.load(f)

MAX_SEQUENCE_LENGTH = 200

MAX_SEQUENCE_LENGTH = 200

print("✅ Model loaded successfully!")
print(f"✅ Vocabulary size: {len(word_to_index):,}")
print(f"✅ Classes: {label_encoder.classes_}")

🤖 LOADING LSTM SENTIMENT CLASSIFIER
✅ Model loaded successfully!
✅ Vocabulary size: 20,000
✅ Classes: ['negative' 'neutral' 'positive']


In [15]:
def predict_sentiment(text):
    """
    Predict sentiment of a text review
    
    Args:
        text (str): Review text
        
    Returns:
        dict: Prediction results with sentiment and probabilities
    """
    # Clean text (basic cleaning - adjust to match your preprocessing)
    text_clean = text.lower()
    text_clean = re.sub(r'[^\w\s]', '', text_clean)  # Remove punctuation
    text_clean = re.sub(r'\s+', ' ', text_clean).strip()  # Remove extra spaces
    
    # Tokenize
    tokens = text_clean.split()
    
    # Convert to sequence
    sequence = [word_to_index.get(token, 1) for token in tokens]  # 1 = <OOV>
    
    # Pad sequence
    padded = pad_sequences([sequence], maxlen=MAX_SEQUENCE_LENGTH, padding='post', value=0)
    
    # Predict
    prediction = model.predict(padded, verbose=0)[0]
    
    # Get predicted class and confidence
    predicted_class_idx = np.argmax(prediction)
    predicted_sentiment = label_encoder.classes_[predicted_class_idx]
    confidence = prediction[predicted_class_idx] * 100
    
    # Return results
    return {
        'sentiment': predicted_sentiment,
        'confidence': confidence,
        'probabilities': {
            'negative': prediction[0] * 100,
            'neutral': prediction[1] * 100,
            'positive': prediction[2] * 100
        }
    }

In [ ]:
print("\n" + "=" * 80)
print("🧪 TESTING WITH EXAMPLES")
print("=" * 80)

test_reviews = [
    "This restaurant is absolutely amazing! The food was delicious and service was perfect.",
    "Terrible experience. The food was cold and the staff was rude. Never coming back!",
    "It was okay, nothing special. Average food and service.",
    "I love this place! Best pizza in town. Highly recommended!",
    "Worst meal ever. Disgusting and overpriced."
]

for i, review in enumerate(test_reviews, 1):
    result = predict_sentiment(review)
    
    print(f"\n📝 Review {i}:")
    print(f"   Text: {review[:80]}...")
    print(f"   🎯 Sentiment: {result['sentiment'].upper()}")
    print(f"   💯 Confidence: {result['confidence']:.1f}%")
    print(f"   📊 Probabilities:")
    print(f"      Negative: {result['probabilities']['negative']:.1f}%")
    print(f"      Neutral:  {result['probabilities']['neutral']:.1f}%")
    print(f"      Positive: {result['probabilities']['positive']:.1f}%")


🧪 TESTING WITH EXAMPLES

📝 Review 1:
   Text: This restaurant is absolutely amazing! The food was delicious and service was pe...
   🎯 Sentiment: POSITIVE
   💯 Confidence: 99.1%
   📊 Probabilities:
      Negative: 0.0%
      Neutral:  0.9%
      Positive: 99.1%

📝 Review 2:
   Text: Terrible experience. The food was cold and the staff was rude. Never coming back...
   🎯 Sentiment: NEGATIVE
   💯 Confidence: 99.7%
   📊 Probabilities:
      Negative: 99.7%
      Neutral:  0.3%
      Positive: 0.0%

📝 Review 3:
   Text: It was okay, nothing special. Average food and service....
   🎯 Sentiment: NEUTRAL
   💯 Confidence: 83.7%
   📊 Probabilities:
      Negative: 15.5%
      Neutral:  83.7%
      Positive: 0.7%

📝 Review 4:
   Text: I love this place! Best pizza in town. Highly recommended!...
   🎯 Sentiment: POSITIVE
   💯 Confidence: 99.6%
   📊 Probabilities:
      Negative: 0.0%
      Neutral:  0.4%
      Positive: 99.6%

📝 Review 5:
   Text: Worst meal ever. Disgusting and overpriced....
 

In [32]:
review = input()

result = predict_sentiment(review)

print(f"\n📝 Review:")
print(f"   Text: {review[:80]}...")
print(f"   🎯 Sentiment: {result['sentiment'].upper()}")
print(f"   💯 Confidence: {result['confidence']:.1f}%")
print(f"   📊 Probabilities:")
print(f"      Negative: {result['probabilities']['negative']:.1f}%")
print(f"      Neutral:  {result['probabilities']['neutral']:.1f}%")
print(f"      Positive: {result['probabilities']['positive']:.1f}%")


📝 Review:
   Text: the food gave me diarrea...
   🎯 Sentiment: NEGATIVE
   💯 Confidence: 61.4%
   📊 Probabilities:
      Negative: 61.4%
      Neutral:  24.5%
      Positive: 14.2%
